# Motivation for Ansible
## Automating Device Management
- It all started with servers in the server room and administrator managing it
- The server room become a datacenter
- And after cloud came Internet of Things
- How are you going to manage that is a consistent way?

## Ansible vs. Shell Scripts
- Modern day datacenters need automation
- Shell scripting is feasible in small environments only
- - Can you install 3 servers as a webserver with a database using shell scripts?
- - How easy is it to put different parameters in shell scripts?
- - What can you do with a shellscript if and administrator makes an error and removes configuration?
- - How easy is it to use a shell script deploying a desired state?
- - What can you od with shell scripts across different Linux Distributions?
- The answer lies in Configuration Management
- In Configuration Management, you'll define a desired state. If current state changes, you'll just re-apply the desired state.

## Ansible vs. Puppet and Others
- Ansible is just one of the configuration managemnet solutions
- Other solutions are Puppet, Chef, Salt, CFEngine and more
- Is Ansible really better than the others?
- It is definitely easier than many others
- And it doesn't need an agent on manged servers, but uses SSH
- Ansible is modular which makes it fexible
- And the modules by defult written in Python
- Ober 1000 modules are already available and administrators who know Pyothon can develop their own Ansible modules
 
## Beyond just Linux
- Using Ansible allows you to manage anything
- The origins were in managing Linux through SSH
- Using different plugins, many other assets can be managed
- Different assets can be managed by direct API access
- In the end that helps managing the enterprise through Ansible
- Modules are written for very differetn assets, to speak the language of the specific managed devices

# Create Virtual-Machines
- Install VirtualBox, and set up 3 Virtual Machines
- - Controller Virtual Machine
- - Node1 Virtual Machine
- - Node2 Virtual Machine
- Follow the instruction to create 3 CEntOS 7.3 Virtual Machine from the "Virtual Machine" Tutorial in this repository

# Set up SSH communication between Virtual Machines
- First add the Hostnames and IP address of our virtual machines, to the local DNS of our virtual machine by editing the **/etc/hosts** file

### Configuring SSH
- Set up SSH Key-based authentication 
- - **ssh-keygen**
- This creates a public key as well as a private key
- - The server that has the public key sends a challenge that can only be answered with the private key
- - Keey the private key in the local user account on the control node
- - Send the publkic key to the **~/.ssh/authorized_keys** file in the target user home directory
- - - Use **ssh-copy-id user@remotehost**
- - - Notice that the lodcal user name and the remote user name do NOT have to be the same
- Don't forget to include the controller host as well if you want to manage that also

Add the user to wheel, in every VM run

In [ ]:
%%bash
su -
visudo # check if wheel in enabled
usermod -aG wheel controller # run in controller vm
usermod -aG wheel node1 # run in node1 

Restart all the VMs

Now make sure that all the VMs are up
- SSH into the controller VM

In [ ]:
%%bash
# in controller VM
su -
vi /etc/hosts

Enter the configurations in the hosts file

 Copy the hosts file to other vms

In [ ]:
%%bash
# in controller VM
scp /etc/hosts node1:/etc/
scp /etc/hosts node2:/etc/

Setup SSH

In [ ]:
%%bash
# in controller VM
su controller
ssh-keygen # just use default settings
ssh-copy-id node1@node1.example.com
ssh-copy-id node2@node2.example.com

### Now we can ssh into node1 and node2 from controller without being prompt for password or passphrase

In [ ]:
%%bash
# in controller VM
ssh node1@node1
ssh node2@node2

# Installing Ansible
## Prerequisite
- You will need a minimum of one Controller node and one Managed host
- - All is focused on Linux managed machines
- Install Python 2
- Add the EPEL (Extra Packages for Enterprise Linux) repository, EPEL repository is managed by the EPEL group.
- The EPEL group creates, maintains, and mnages a high quality set of additional packages, that is not included in the core repository. EPEL repository is a part of the non-core repositories of Red Hat distros
- Install Ansible from EPEL
- Create a non-root user and perform all Ansible tasks as non-root user

## Installing Managed Nodes
- Same requires as the controller node

On all of the VMs run

In [ ]:
%%bash
# on all VMs
su -
yum install -y python2 epel-release
yum install -y ansible

# Inventory File
Managing Managed Hosts
- After installation, you can use the **ansible** command agains remote hsots
- Remote hosts need to pe sepcified in the inventory file
- The inventory file allows you to defined managed hosts
- Hosts are specified by their FQDN (Fully Qualified Domain Name) or IP address
- Hosts may be mentioned more than once
- - This allows you to create logical groups
- - A host may belong to multiple logical groups
- In **ansible** commands, you'll mention host names, as well as the inventory file that you're going to use
- - **ansible server1.example.com, server2.example.com -i myinventory --list-hosts**

## Inventory File Location
- The inventory file is indidcated with the **-i** option
- Typically, you can create an Ansible project directory in your home directory, and put an inventory file in there

## Creating Inventory File
- In controller VM

In [ ]:
%%bash
# in controller VM
mkdir ~/install
cd ~/install
touch inventory
vi inventory

### The inventory file 
- Identifies all the hosts that ansible can manage
- And put all the hosts in a group called "all"

In [ ]:
%%bash
# in controller VM
ansible all -i inventory --list-hosts

# Ansible Configuration File
## Managin the Ansible Configuration File
- There is no single ansible configuration file, it can be in multiple locations
- The ansible.cfg file specifies how Ansible should be used and can be f## Ad-hoc Commandsound in different locations
- - The generic file /etc/ansible/ansible.cfg
- - The user specific file ~/.ansible.cfg
- It's common practice to use ans ansible.cfg file in the project directory
- Alternatively, specify \\$ANSIBLE_CONFIG environment variable
- The ansible.cfg file that is used should contain all environment variables
- User **ansible -v** to find out which configuration file is used 

## ansible.cfg Contents
- become: specifies how to escalate privileges to managed hosts, eg whether sudo should be used to escalate priviledges
- become_user: specifies which user account to use on the remote host
- become_ask_pass: whether or not a password should be asked for
- inventory: which inventory file to use
- remote_user: name of the user account on the managed machine
- - Not set by default, resulting in the local user name being used

## Previledge Escalation
- Ansible runs tasks on the manageed hosts with the same user account as the local user
- - So make sure tha SSH keys are copied to that user's SSH config on remote user
- Set **remote_user** in ansible.cfg to specify another user to be used
- If **remote_user** is not specified, privilege escalation can be used
- Enable in the [privilege_escalation] section in ansible.cfg
- - become=True
- - become_method=sudo
- - become_user=root
- - become_ask_pass=False

## Copnfiguring sudo for Privilege Escalation
- Privilege escalation needs a sudo configuration
- For the control node Ansible default account, create a sudo file on all Ansible managed hosts:

# Ad-hoc Commands
Ad-hoc commands are just ansible commands that you run from the command line
- You'll typically want to create playbooks to automate tasks against multiple Ansible servers
- To quickly make changes to mansy managed hosts, ad-hoc commands are convenient
- Ad-hoc commands can also be used for diagnostic purposes, like querying a large number of hosts
- In ad-hoc commands, modules are typoically used

## Modules
- A module is used to accomplish specific tasks in Ansible
- - Written in python an run their own specific arguments
- Modules can run with their own specific arguments
- Modules are specified with the **-m** option, module areguments are referred to with the **-a** option
- The default module can be set in ansible.cfg. It's predefined to the **command** module
- - This module allows you to run random commands against managed hosts
- - As commands is the default module, it doesn't have to be referred to using **-m** module
- - Notice that the command omodule is not interpreted by the shell on the managed host and for that reason cannot work with variables, pipes and redirects
- - Consider using the **shell** module if you need full shell functionality

## 3 Common Modules
- **command**: runs a command on a managed host
- **shell**: runs a command on managed host through the local shell
- **copy**: copy a file, change content on a remote host in a target file

## Ad-hoc Command Examples
- **ansible all -m command -a id**
- - Runs the command module with the **id** command as its argument against all hosts. Notice that this needs [all] to be defined in the inventory
- - ansible all refers to all hosts, specified in the inventory
- - -m command refers to the default module
- - -a id refers to the argument we want to pass to the command module
- **ansible all -m command -a id -o**
- - Same command, but provide a single line of output
- **ansible all -m command -a env**
- - Fails, as the command module doesn't work through the shell, it is an internal command
- **ansible all -m shell -a env**
- **ansible managed1.ansible.local -m copy -a 'content="Ansible managed\n" dest=/etc/motd'**
- - it is a copy command

In [ ]:
%%bash
cd ~/install
ansible all -m command -a id

This doesn't work because we haven't specified the inventory file

In [ ]:
%%bash
cd ~/install
ansible all -i inventory -m command -a id

controller will be unreachable because ssh not install key not setup on controller

Run to following command to install ssh key

In [ ]:
%%bash
ssh controller@controller.example.com
ssh-copy-id controller@controller.example.com

In [ ]:
%%bash
cd ~/install
ansible all -i inventory -m command -a id
ansible all -i inventory -m command -a id -o
ansible all -i inventory -m shell -a env

In [ ]:
%%bash
cd ~/install
ansible node1@node1.example.com -i inventory -m copy -a 'content="Managed by Ansible\n" dest=/etc/motd'

Will throw this error

node1@node1.example.com | FAILED! => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/bin/python"
    }, 
    "changed": false, 
    "checksum": "4458b979ede3c332f8f2128385df4ba305e58c27", 
    "msg": "Destination /etc not writable"
}

because we need to escalate priviledge and not ass for password

First we need to make an ansible.cfg file that escalate priviledges

In [ ]:
%%bash
cd ~/install
vi ansible.cfg

Then run

In [ ]:
%%bash
cd ~/install
ansible node1@node1.example.com -i inventory -m copy -a 'content="Managed by Ansible\n" dest=/etc/motd'

Manage by Ansible is copied to /etc/motd file in node1

## If We want to Run commands on managed hosts
we need to make sure confutration is set up properly
- **ansible.cfg** file is set up correctly
- **inventory** file is set up correctly
- and appropriate **sudo** setup as well

# Adding a Managed Host Summary
- Make sure the ansible user account that you're using exists
- Copy ssh keys for the ansible user account to the remote host
- Log in once, using SSH so that the remote host publick key is stored on the control host
- Create sudo configuration
- Install pythong and ansibole packages
- Update the inventory file